In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

C:\Users\guilh\AppData\Local\Temp\ipykernel_15376\2080034654.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
medicoes2 = pd.read_csv('./04-dados-qualar-longo-AML/medicoes-longo_seleccionadas_AML.csv', thousands=',', index_col=0, parse_dates=True)
medicoes2['Dia']= medicoes2.index.day #adicionar coluna com o Dia
medicoes2['Ano']=medicoes2.index.year #adicionar coluna com o Ano
medicoes2['Mes']=medicoes2.index.month #adicionar coluna com o Mes
medicoes2

,Station,Poluente,Valor,Dia,Ano,Mes
data,,,,,,
2001-01-01 00:00:00,Alfragide/Amadora,SO2,2.9,1,2001,1
2001-01-01 01:00:00,Alfragide/Amadora,SO2,2.9,1,2001,1
2001-01-01 02:00:00,Alfragide/Amadora,SO2,5.6,1,2001,1
2001-01-01 03:00:00,Alfragide/Amadora,SO2,5.6,1,2001,1
2001-01-01 04:00:00,Alfragide/Amadora,SO2,5.6,1,2001,1
...,...,...,...,...,...,...
2022-12-31 19:00:00,Paio Pires,PM2.5,8.8,31,2022,12
2022-12-31 20:00:00,Paio Pires,PM2.5,10.5,31,2022,12
2022-12-31 21:00:00,Paio Pires,PM2.5,9.1,31,2022,12


In [3]:
#descobrir os valores do AQI por dia
df_PM10_PM25=medicoes2[medicoes2['Poluente'].isin(['PM10', 'PM2.5'])].groupby(['Ano','Mes','Dia', 'Station', 'Poluente']).mean() #media PM10 e PM2.5
df_SO2_NO2_O3=medicoes2[medicoes2['Poluente'].isin(['SO2', 'NO2', 'O3'])].groupby(['Ano','Mes','Dia', 'Station', 'Poluente']).max() #máximo SO2, NO2 e O3
medicoes3 = pd.merge(medicoes2, df_PM10_PM25.reset_index(),on=['Ano', 'Mes', 'Dia','Station','Poluente'], how='left') #fazer os merge os valores ficam em Valor_y
medicoes3 = pd.merge(medicoes3, df_SO2_NO2_O3.reset_index(),on=['Ano', 'Mes', 'Dia','Station','Poluente'], how='left') #fazes os merge os valores ficam em Valor

medicoes3['Valor_y'] = medicoes3['Valor_y'].fillna(0)
medicoes3['Valor'] = medicoes3['Valor'].fillna(0) #substituir os NAN por 0
medicoes3['Valores dos AQIs']=medicoes3['Valor_y']+medicoes3['Valor'] #juntar as duas colunas com os valores do AQI

In [4]:
#calcular o AQI geral do dia 
AQI_geral_do_dia=medicoes3.groupby(['Ano', 'Mes','Dia', 'Station'])['Valores dos AQIs'].max()
medicoes3=pd.merge(medicoes3, AQI_geral_do_dia.reset_index(), on=['Ano', 'Mes', 'Dia','Station'], how='left')
medicoes3

,Station,Poluente,Valor_x,Dia,Ano,Mes,Valor_y,Valor,Valores dos AQIs_x,Valores dos AQIs_y
0,Alfragide/Amadora,SO2,2.9,1,2001,1,0.000000,8.2,8.200000,78.0
1,Alfragide/Amadora,SO2,2.9,1,2001,1,0.000000,8.2,8.200000,78.0
2,Alfragide/Amadora,SO2,5.6,1,2001,1,0.000000,8.2,8.200000,78.0
3,Alfragide/Amadora,SO2,5.6,1,2001,1,0.000000,8.2,8.200000,78.0
4,Alfragide/Amadora,SO2,5.6,1,2001,1,0.000000,8.2,8.200000,78.0
...,...,...,...,...,...,...,...,...,...,...
10379329,Paio Pires,PM2.5,8.8,31,2022,12,8.216667,0.0,8.216667,72.0
10379330,Paio Pires,PM2.5,10.5,31,2022,12,8.216667,0.0,8.216667,72.0
10379331,Paio Pires,PM2.5,9.1,31,2022,12,8.216667,0.0,8.216667,72.0
10379332,Paio Pires,PM2.5,7.0,31,2022,12,8.216667,0.0,8.216667,72.0


In [5]:
#Limpar um pouco o dataframe 
medicoes_AQI=medicoes3.copy()
medicoes_AQI.set_index(medicoes2.index, inplace=True)
medicoes_AQI.drop(columns=['Valor_y','Valor'], inplace=True)
medicoes_AQI.rename(columns={'Valor_x':'Valor', 'Valores dos AQIs_x':'AQI ppl', 'Valores dos AQIs_y':'AQI pd' }, inplace=True)
medicoes_AQI['AQI ppl']=medicoes_AQI['AQI ppl'].round(1) #arredondar para uma casa decimal para ficar com uma compreensão mais fácil
medicoes_AQI

,Station,Poluente,Valor,Dia,Ano,Mes,AQI ppl,AQI pd
data,,,,,,,,
2001-01-01 00:00:00,Alfragide/Amadora,SO2,2.9,1,2001,1,8.2,78.0
2001-01-01 01:00:00,Alfragide/Amadora,SO2,2.9,1,2001,1,8.2,78.0
2001-01-01 02:00:00,Alfragide/Amadora,SO2,5.6,1,2001,1,8.2,78.0
2001-01-01 03:00:00,Alfragide/Amadora,SO2,5.6,1,2001,1,8.2,78.0
2001-01-01 04:00:00,Alfragide/Amadora,SO2,5.6,1,2001,1,8.2,78.0
...,...,...,...,...,...,...,...,...
2022-12-31 19:00:00,Paio Pires,PM2.5,8.8,31,2022,12,8.2,72.0
2022-12-31 20:00:00,Paio Pires,PM2.5,10.5,31,2022,12,8.2,72.0
2022-12-31 21:00:00,Paio Pires,PM2.5,9.1,31,2022,12,8.2,72.0


In [6]:
#função que traduz a tabela das categorias
def escala_AQI(pl, valor):
    if pl=='PM10':
        if 0 <= valor <= 20:
            return 1  # Muito bom
        elif 21 <= valor <= 35:
            return 2  # Bom
        elif 36 <= valor <= 50:
            return 3  # Médio
        elif 51 <= valor <= 100:
            return 4  # Fraco
        elif 101 <= valor <= 1200:
            return 5  # Mau
    elif pl=='PM2.5':
        if 0 <= valor <= 10:
            return 1
        elif 11 <= valor <= 20:
            return 2
        elif 21 <= valor <= 25:
            return 3
        elif 26 <= valor <= 50:
            return 4
        elif 51 <= valor <= 800:
            return 5
    elif pl=='NO2':
        if 0 <= valor <= 40:
            return 1
        elif 41 <= valor <= 100:
            return 2
        elif 101 <= valor <= 200:
            return 3
        elif 201 <= valor <= 400:
            return 4
        elif 401 <= valor <= 1000:
            return 5
    elif pl=='O3':
        if 0 <= valor <= 80:
            return 1
        elif 81 <= valor <= 100:
            return 2
        elif 101 <= valor <= 150:
            return 3
        elif 151 <= valor <= 240:
            return 4
        elif 241 <= valor <= 600:
            return 5
    elif pl=='SO2':
        if 0 <= valor <= 100:
            return 1
        elif 101 <= valor <= 200:
            return 2
        elif 201 <= valor <= 350:
            return 3
        elif 351 <= valor <= 500:
            return 4
        elif 501 <= valor <= 1250:
            return 5


In [7]:
#adicionar coluna com a categoria
medicoes_AQI['escala'] = medicoes_AQI.apply(lambda row: escala_AQI(row['Poluente'], row['AQI ppl']), axis=1)
medicoes_AQI

,Station,Poluente,Valor,Dia,Ano,Mes,AQI ppl,AQI pd,escala
data,,,,,,,,,
2001-01-01 00:00:00,Alfragide/Amadora,SO2,2.9,1,2001,1,8.2,78.0,1.0
2001-01-01 01:00:00,Alfragide/Amadora,SO2,2.9,1,2001,1,8.2,78.0,1.0
2001-01-01 02:00:00,Alfragide/Amadora,SO2,5.6,1,2001,1,8.2,78.0,1.0
2001-01-01 03:00:00,Alfragide/Amadora,SO2,5.6,1,2001,1,8.2,78.0,1.0
2001-01-01 04:00:00,Alfragide/Amadora,SO2,5.6,1,2001,1,8.2,78.0,1.0
...,...,...,...,...,...,...,...,...,...
2022-12-31 19:00:00,Paio Pires,PM2.5,8.8,31,2022,12,8.2,72.0,1.0
2022-12-31 20:00:00,Paio Pires,PM2.5,10.5,31,2022,12,8.2,72.0,1.0
2022-12-31 21:00:00,Paio Pires,PM2.5,9.1,31,2022,12,8.2,72.0,1.0


In [8]:
medicoes_AQI.to_csv('./04-dados-qualar-longo-AML/indice_qualidade_ar_diario.csv', index=True) #guardar o dataframe